# 🧠 Laboratorio Multimodal con Oracle Generative AI Inference

## Introducción

En este laboratorio aprenderás a utilizar el **servicio Oracle Generative AI Inference** para realizar una interacción **multimodal**: enviar texto junto con una imagen a un modelo de lenguaje de OCI y recibir una respuesta generada de manera contextual.

El objetivo es demostrar, de forma práctica y simple, cómo **integrar capacidades de visión y lenguaje natural (VLM)** dentro de un flujo de Python utilizando el SDK oficial de Oracle.

A través de este ejemplo podrás:

* Convertir una imagen local en una cadena Base64 válida para el modelo.
* Enviar un mensaje tipo *chat* que combina texto e imagen.
* Obtener la descripción generada por el modelo directamente desde OCI.
* Comprender la estructura de objetos `ChatDetails`, `GenericChatRequest` y `Message` dentro del SDK.

## Requisitos previos

Antes de ejecutar el laboratorio, asegúrate de contar con:

* Un **modelo Generative AI** disponible en tu tenancy (modalidad *On-Demand* o *Deployment*).
* Credenciales configuradas en tu archivo `~/.oci/config`. (configurado previamente en el lab01)
* Variables definidas:

  ```python
  ENDPOINT = "https://inference.generativeai.<region>.oci.oraclecloud.com" # NO Modificar
  MODEL_ID = "<OCID_DEL_MODELO>" # NO Modificar
  COMPARTMENT_ID = "<OCID_DEL_COMPARTMENT>" # Modificar
  ```
* Librerías instaladas:

  ```bash
  pip install oci
  ```

## Objetivo del ejemplo

Enviar una imagen junto con una instrucción textual para que el modelo describa su contenido, demostrando el potencial de la IA generativa en **aplicaciones con comprensión visual y textual**.


In [ ]:
# --- Multimodal chat (imagen + texto) en OCI Generative AI---

import os, json, base64, mimetypes
import oci


## Debemos actualizar los datos del compartment para la ejecución del código

In [ ]:
# === Configuración base ===
CONFIG_PROFILE = "DEFAULT"
config = oci.config.from_file(os.path.expanduser("~/.oci/config"), CONFIG_PROFILE)
config["region"] = "us-chicago-1"

##datos actualizar
COMPARTMENT_ID = "ocid1.compartment.oc1.."
ENDPOINT = "https://inference.generativeai.us-chicago-1.oci.oraclecloud.com"

MODEL_ID = "ocid1.generativeaimodel.oc1.us-chicago-1.amaaaaaask7dceyayjawvuonfkw2ua4bob4rlnnlhs522pafbglivtwlfzta"
assert ".oc1.us-chicago-1." in MODEL_ID, "El model_id debe ser de la región us-chicago-1"


In [ ]:


# === Utilidad: convertir imagen local a data URL base64 ===
def image_to_data_url(path: str) -> str:
    mime = mimetypes.guess_type(path)[0] or "image/jpeg"
    with open(path, "rb") as f:
        b64 = base64.b64encode(f.read()).decode("utf-8")
    return f"data:{mime};base64,{b64}"

# Ruta de tu imagen local
IMG_PATH = "demo-image.jpeg"
PROMPT = "Describe brevemente lo que ves en la imagen."

# Prepara contenidos multimodales
from oci.generative_ai_inference import GenerativeAiInferenceClient
from oci.generative_ai_inference.models import (
    ChatDetails,
    OnDemandServingMode,
    GenericChatRequest,
    BaseChatRequest,
    Message,
    TextContent,
    ImageContent,
    ImageUrl,
)

# Imagen como data URL (también puedes usar una URL pública HTTPS)
img_data_url = image_to_data_url(IMG_PATH)

image_url = ImageUrl(url=img_data_url)
image_content = ImageContent(image_url=image_url)

text_content = TextContent(text=PROMPT)

user_message = Message(role="USER", content=[text_content, image_content])

chat_request = GenericChatRequest(
    api_format=BaseChatRequest.API_FORMAT_GENERIC,
    messages=[user_message],
    max_tokens=400,
    temperature=0.4,
    top_p=0.9,
)

chat_details = ChatDetails(
    serving_mode=OnDemandServingMode(model_id=MODEL_ID),
    chat_request=chat_request,
    compartment_id=COMPARTMENT_ID,
)

client = GenerativeAiInferenceClient(config=config, service_endpoint=ENDPOINT)
resp = client.chat(chat_details)

# Extrae y muestra el texto de la respuesta
choices = resp.data.chat_response.choices
if choices:
    parts = choices[0].message.content
    texto = "\n".join(getattr(p, "text", "") for p in parts if hasattr(p, "text"))
    print("\n=== RESPUESTA ===\n", texto.strip())
else:
    print("No se generó respuesta.")

## 🧩 Puntos adicionales: ¡Hazlo interactivo!

Ahora que ya probaste el flujo básico, puedes **subir una foto tuya o cualquier imagen divertida** para que el modelo la interprete y responda de forma contextual.

1. 📸 **Carga tu imagen local** (por ejemplo `selfie-divertida.jpeg` o cualquier imagen creativa).
   Asegúrate de colocarla en la misma carpeta del notebook o ajustar la ruta:

   ```python
   IMG_PATH = "selfie-divertida.jpeg"
   ```

2. 💬 **Formula preguntas abiertas** al modelo relacionadas con la imagen.
   Por ejemplo:

   ```python
   PROMPT = """
   Observa la foto y dime qué emociones transmite.
   ¿Qué tipo de actividad parece estar realizando la persona?
   ¿Podrías generar una historia corta inspirada en la imagen?
   """
   ```

3. 🚀 **Ejecuta nuevamente la celda principal del chat**
   Verás cómo el modelo analiza tanto la imagen como el texto, generando respuestas personalizadas según el contenido visual.

4. 🧠 **Experimenta con distintas instrucciones**:

   * “Descríbeme el fondo de la foto con detalle.”
   * “Imagina que esta imagen es parte de una película, ¿cómo se llamaría?”
   * “Convierte la escena en un poema corto.”

5. 🎯 **Objetivo final:** Comprender cómo los modelos multimodales pueden interpretar simultáneamente información visual y textual, y cómo esto se puede aplicar en contextos reales como **análisis de imágenes, storytelling, educación o generación de contenido creativo.**
